# Data Cleaning

Cleaning and parsing of data from the [NBCUniversal Analytics Challenge](http://sc.aisnet.org/conference2018/student-competitions/nbcuniversal-challenge/).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Loading Data

In [2]:
data = pd.read_csv('Data/NBCU-dataLaurel.csv')
data.head()

,imdbid,title,plot,rating,imdb_rating,metacritic,dvd_release,production,actors,imdb_votes,poster,director,release_date,runtime,genre,awards,keywords,Budget,Box Office Gross
0,tt0010323,The Cabinet of Dr. Caligari,"Hypnotist Dr. Caligari uses a somnambulist, Ce...",UNRATED,8.1,NaN,15-Oct-97,Rialto Pictures,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...","42,583",https://images-na.ssl-images-amazon.com/images...,Robert Wiene,19-Mar-21,67 min,"Fantasy, Horror, Mystery",1 nomination.,expressionism|somnambulist|avant-garde|hypnosi...,18000,0
1,tt0052893,Hiroshima Mon Amour,A French actress filming an anti-war film in H...,NOT RATED,8.0,NaN,24-Jun-03,Rialto Pictures,"Emmanuelle Riva, Eiji Okada, Stella Dassas, Pi...","21,154",https://images-na.ssl-images-amazon.com/images...,Alain Resnais,16-May-60,90 min,"Drama, Romance",Nominated for 1 Oscar. Another 6 wins & 5 nomi...,memory|atomic-bomb|lovers-separation|impossibl...,88300,0
2,tt0058898,Alphaville,A U.S. secret agent is sent to the distant spa...,NOT RATED,7.2,NaN,20-Oct-98,Rialto Pictures,"Eddie Constantine, Anna Karina, Akim Tamiroff","17,801",https://images-na.ssl-images-amazon.com/images...,Jean-Luc Godard,5-May-65,99 min,"Drama, Mystery, Sci-Fi",1 win.,dystopia|french-new-wave|satire|comic-violence...,220000,46585
3,tt0074252,"Ugly, Dirty and Bad",Four generations of a family live crowded toge...,NaN,7.9,NaN,1-Nov-16,Compagnia Cinematografica Champion,"Nino Manfredi, Maria Luisa Santella, Francesco...","5,705",https://images-na.ssl-images-amazon.com/images...,Ettore Scola,23-Sep-76,115 min,"Comedy, Drama",1 win & 2 nominations.,incest|failed-murder-attempt|poisoned-food|bap...,6590,0
4,tt0084269,Losing Ground,A comedy-drama about a Black American female p...,NaN,6.3,NaN,NaN,Milestone Film & Video,"Billie Allen, Gary Bolling, Clarence Branch Jr...",132,https://images-na.ssl-images-amazon.com/images...,Kathleen Collins,1-Jun-82,86 min,"Comedy, Drama",NaN,artist|painter|marriage|black-independent-film...,0,0


In [3]:
data.shape

(8468, 19)

These are the variables we have to work with:

imdbid: Unique Id used by IMDB to refer to the movie.

Title: Title of the movie

plot: Movie plot summary

rating: MPAA Appropriate audience rating

imdb_rating: IMDB's voters' scoring of a movie on a scale from 1-10 (10 being best)

metacritic: Metacritic movie score on a scale of 0-100 (100 being best)

dvd_release: Movie release date on DVD

production: Principle production company

actors: Lead Actors

imdb_votes: Total votes from IMDB members

poster: Movie Poster artwork

director: Movie director

release_date: Theatrical Release Date

runtime: Runtime length of movie in minutes

genre: Genre Classification

awards: Academy awards & nominations

keywords: Keywords associated with the movie

budget: Budget spent on movie production, marketing, and distribution

box office gross: Box Office Gross Returns as of 9/21/2017

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null object
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null object
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: float64(2), object(17)
memory usage: 1.2+ MB


Notice how many of variables are just objects. We're going to have to deal with converting a few of these into useful types.

### Parsing release_dates and dvd_release

First, we'll start by changing release_date to a datetime-like type.

In [5]:
data['release_date'].head()

0    19-Mar-21
1    16-May-60
2     5-May-65
3    23-Sep-76
4     1-Jun-82
Name: release_date, dtype: object

In [6]:
pd.to_datetime(data['release_date'])[1],data['release_date'][1]

(Timestamp('2060-05-16 00:00:00'), '16-May-60')

Then we see that there is an issue with pandas to_datetime function. It converts very old dates back to the 19th century. Perhaps we need to use the datetime package per [this](https://stackoverflow.com/questions/16600548/how-to-parse-string-dates-with-2-digit-year).

In [7]:
dates = data['release_date']
dates = pd.to_datetime(dates)
opivot_index = dates[dates.apply(lambda x: x.year>2019)].index
for index in opivot_index:
    dates[index] = dates[index].replace(year = dates[index].year-100)
sum(dates>'2019-12-31')

0

Then we've found a way to account for Python's default pivot year.

In [8]:
data['release_date']=dates
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null object
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](1), float64(2), object(16)
memory usage: 1.2+ MB


It seems natural to also do the same for dvd_release.

In [9]:
dates = data['dvd_release']
dates = pd.to_datetime(dates)
opivot_index = dates[dates.apply(lambda x: x.year>2019)].index
for index in opivot_index:
    dates[index] = dates[index].replace(year = dates[index].year-100)
data['dvd_release'] = dates
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null datetime64[ns]
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](2), float64(2), object(15)
memory usage: 1.2+ MB


### Parsing imdb_votes

Next, we have several important numerical variables that are currenly in object types. First, we'll work with imdb_votes.

In [10]:
data['imdb_votes'].head()

0    42,583
1    21,154
2    17,801
3     5,705
4       132
Name: imdb_votes, dtype: object

So we need to convert imdb_votes to integers. Since there are commas in each number, we cannot simply tell pandas to treat each entry as an integer via the .astype() function. We'll first have to replace each comma with a blank, then apply the int() function. We also have to take care to ignore all of the missing values from imdb_votes as we will be dealing with those later.

In [12]:
votes = data['imdb_votes']
votes_parse = votes.str.replace(',','')
votes_parse.head()

0    42583
1    21154
2    17801
3     5705
4      132
Name: imdb_votes, dtype: object

Then, we've successfully removed all of the commas. Let's confirm that we didn't lose any datapoints along the way.

In [13]:
[len(votes_parse), len(data['imdb_votes'])]

[8468, 8468]

In order to convert to int, we have to find a way to work around missing values. Let's replace all the missing values with -1 and then convert them back to NaN after conversion.

In [14]:
import numpy as np

votes_int = votes_parse.fillna(-1).astype('int')
votes_int[votes_int==-1] = np.nan
votes_int.isna().sum() == votes_parse.isna().sum()

True

In [15]:
data['imdb_votes'] = votes_int
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null datetime64[ns]
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null float64
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](2), float64(3), object(14)
memory usage: 1.2+ MB


### Parsing Budget and Box Office Gross

Now we have to deal with 'Budget' and 'Box Office Gross' in a similar manner.

In [16]:
data[['Budget', 'Box Office Gross']].head()

,Budget,Box Office Gross
0,18000,0
1,88300,0
2,220000,46585
3,6590,0
4,0,0


Looks like we don't have to worry about any commas in 'Budget' or in 'Box Office Gross', so the conversions will be much simplier. In the previous revision of Data Cleaning, we found that data entries containing Box Office Gross values in foreign currencies were not worth saving. We will be removing them in this revision.

In [23]:
data = data.drop(data[data['Box Office Gross'].str.contains('GBP')].index)
data = data.drop(data[data['Box Office Gross'].str.contains('EU')].index)
data = data.drop(data[data['Budget'].str.contains('EU')].index)
data = data.drop(data[data['Budget'].str.contains('CAD')].index)

Now that we've removed all datapoints with foreign currencies, we can start parsing Budget and Box Office Gross into numerical types.

In [36]:
gross = data['Box Office Gross']
gross = gross.str.replace(',','')

for i, item in enumerate(gross):
    try:
        int(item)
    except ValueError:
        print('ERROR at index {}:{!r}'.format(i, item))

ERROR at index 1394:'<strongitemprop="name">Grossbuttotallyworthwhile</strong>'
ERROR at index 1869:'<pitemprop="reviewBody">HumanTraffickingbywayofanInternetScam&#x22;Selling&#x22;ChildrentoDesperateWantingRichParentsTryingtoAvoidRed-TapewiththePretenseofLegitimacy.<br/><br/>TheLow-BudgetMoviestartswiththeDevastationofaHaitianEarthquakeanditsAftermath.TheOrphanedChildrenarethenExploitedasProduct.<br/><br/>PuertoRicanLocationsHighlighttheBackdroptowhatamountstoaFranticChaseto&#x22;Reclaim&#x22;aLittleGirlthathasbeen&#x22;Reclaimed&#x22;bytheTraffickers.It&#x27;saBattleofBrawnwithveryLittleWitonBothSides.<br/><br/>JohnCusacvsRyanPhillippeandRachelleLefevrewithLittleBrianaRoyasthe&#x22;Daughter&#x22;/Package.<br/><br/>It&#x27;sSuspensefulEnoughwiththeActionandActingonParwiththeProductionEspeciallyCusacwhoElevatestheProceedingsinquiteaSlimyRoleinthisFormulaicFilm.<br/><br/>WorthaWatchforInterestedViewersbutdon&#x27;texpectanySurprisesItdoeshavearatherDistantFeelconsideringtheUpCloseSituat

In [40]:
gross[1869]

'0'